# Clinical Terminology Detection with AWS Services

In this example we will use AWS Services to convert clinical notes into SNOMED CT classifications. The steps are as follows:

1. Comprehend Medical identifies the clinical entities in the text
2. These entites are converted to SNOMED idenitifiers using the SNOMED website.

This is a python adaptation of the ideas described in the blogpost and associated code:
https://aws.amazon.com/blogs/machine-learning/map-clinical-notes-to-the-omop-common-data-model-and-healthcare-ontologies-using-amazon-comprehend-medical/

Note: After investigation it was determined that this example no longer functions properly due to changes in the SNOMED browser APIs.


In [15]:
import pandas as pd
import numpy as np
import sagemaker
import boto3
import glob
from decimal import *

boto_session = boto3.Session()
region = boto_session.region_name
sgmk_session = sagemaker.Session()
sgmk_role = sagemaker.get_execution_role()

# Access Comprehend Medical

Access to comprehend medical will require an IAM user role with specific policy added to permit the API. In this example we have used the AWS managed policy: [ComprehendMedicalFullAccess](https://console.aws.amazon.com/iam/home#/policies/arn%3Aaws%3Aiam%3A%3Aaws%3Apolicy%2FComprehendMedicalFullAccess). However, you can add a custom or inline policy to permit access to certain API endpoint only by following [these instructions](https://docs.aws.amazon.com/comprehend/latest/dg/access-control-managing-permissions-med.html).

In [ ]:
comprehend = boto3.client('comprehend', region_name=region)

In [18]:
def call_comprehend_medical(note_text):
    client = boto3.client(service_name='comprehendmedical', region_name=region)
    result = client.detect_entities(Text = note_text)
    entities = result['Entities'];
    return entities

# Examples

Here are three examples taken from the synthetic nursing handover dateset.

In [19]:
handovers =  []

In [20]:
handovers.append("Vera Abbott,93, bed four, under Dr Liu came in with chest pain with a history of stroke and previous chest pains,asthma,cataract and glaucoma.almost blind and needs assistance.had 3 nitros with no effect.still under monitoring")

In [21]:
handovers.append("Bed eight, Michael I Wu. Forty-eight years under Dr Hanlen. He came in with headache and vertigo. He's got a history of headache, tinnitus, Bell's Palsy to the left side of his face. That's where his headache has been for the last three years. He's also got photophobia. His GCS is 15 pupils equal and reactive. He's just came back from a brain MRI in Woden. He's ambulant and self-caring but he's a little bit unsteady at times. OBS are stable. He is for carotid doppler, he was supposed to have this morning at 950 but that pushed it back to 1050. Hmmm. 1030, sorry. Because they were late. Then the team were here and they said it's cutting it too close to his MRI so he needs another carotid doppler appointment. Other than that Mike is fine.")

In [22]:
handovers.append("Yunita Muleta, bed 7,  51 years old under Dr Garcia, came in with heartburn under investigation.obs stable.nil other issues")

In [24]:
 ents = call_comprehend_medical(handovers[0])

In [25]:
ents

[{'Id': 9,
  'BeginOffset': 0,
  'EndOffset': 11,
  'Score': 0.9181470274925232,
  'Text': 'Vera Abbott',
  'Category': 'PROTECTED_HEALTH_INFORMATION',
  'Type': 'NAME',
  'Traits': []},
 {'Id': 10,
  'BeginOffset': 35,
  'EndOffset': 38,
  'Score': 0.9977637529373169,
  'Text': 'Liu',
  'Category': 'PROTECTED_HEALTH_INFORMATION',
  'Type': 'NAME',
  'Traits': []},
 {'Id': 0,
  'BeginOffset': 52,
  'EndOffset': 57,
  'Score': 0.9450081586837769,
  'Text': 'chest',
  'Category': 'ANATOMY',
  'Type': 'SYSTEM_ORGAN_SITE',
  'Traits': []},
 {'Id': 3,
  'BeginOffset': 52,
  'EndOffset': 62,
  'Score': 0.7490196824073792,
  'Text': 'chest pain',
  'Category': 'MEDICAL_CONDITION',
  'Type': 'DX_NAME',
  'Traits': [{'Name': 'SYMPTOM', 'Score': 0.5308904647827148},
   {'Name': 'DIAGNOSIS', 'Score': 0.46669456362724304}]},
 {'Id': 4,
  'BeginOffset': 81,
  'EndOffset': 87,
  'Score': 0.9930179119110107,
  'Text': 'stroke',
  'Category': 'MEDICAL_CONDITION',
  'Type': 'DX_NAME',
  'Traits': [{'Na

# Process the returned Named Entities

Comprehend medical has produced a set of named entities from the handover text.
We can process these by both type and confidence score.

In [29]:

min_score = 0.8

def acceptable_entity(entity):
    if (entity['Category'] == "PROTECTED_HEALTH_INFORMATION"):
        return False
    if (entity['Score'] < min_score):
        return False
    return True


In [32]:
def process_entities_to_list(entities):
    result = []
    for entity in entities:        
        if acceptable_entity(entity):
            result.append(entity['Text'])
    return result
        

In [33]:
print(process_entities_to_list(ents))

['chest', 'stroke', 'chest', 'chest pains', 'asthma', 'cataract', 'glaucoma']


# Query SNOMED

In this final section we used the R code from the exmaple blog to query SNOMED for the entities that had been identified. 

This section is not functional due to API changes.

In [34]:
import requests

In [35]:
# GET SNOMED CODES
#  REST interface for search the SNOMED ontology
base = "https://browser.ihtsdotools.org/"
endpoint = "api/snomed/en-edition/v20180131/descriptions"
query_params = "&limit=1&searchMode=partialMatching&lang=english&statusFilter=activeOnly&skipTo=0&returnLimit=1&normalize=true"

def get_snomed_query_url(text):
    return base + endpoint +"?query=" + text + query_params

#get_snomed = GET(snomed_call, type = "basic")
#get_snomed_text = content(get_snomed, "text")

In [37]:
get_snomed_query_url('chest')

'https://browser.ihtsdotools.org/api/snomed/en-edition/v20180131/descriptions?query=chest&limit=1&searchMode=partialMatching&lang=english&statusFilter=activeOnly&skipTo=0&returnLimit=1&normalize=true'

In [36]:
response = requests.get( get_snomed_query_url('chest') )
print(response)

<Response [404]>
